In [1]:
from evaluator import evaluate, calc_impact, create_functions_in_db
from definitions import Repository
import db
import smells

if not db.prepare(input("connection string")):
    print("Can't connect to db")

# Ewaluacja

In [2]:
repo = input("repository")

## Udział smelly prs wśród badanych prs

In [3]:
print(evaluate(repo, smells.lack_of_review))
print(evaluate(repo, smells.missing_description))
print(evaluate(repo, smells.large_changesets))
print(evaluate(repo, smells.sleeping_reviews))
print(evaluate(repo, smells.review_buddies))
print(evaluate(repo, smells.ping_pong))
print(evaluate(repo, smells.union,
         [smells.lack_of_review,
          smells.missing_description,
          smells.large_changesets,
          smells.sleeping_reviews,
          smells.review_buddies,
          smells.ping_pong]))
print(evaluate(repo, smells.intersection,
         [smells.lack_of_review,
          smells.missing_description,
          smells.large_changesets,
          smells.sleeping_reviews,
          smells.review_buddies,
          smells.ping_pong]))

Lack of code review           	93.71%
Missing PR description        	24.04%
Large changeset               	6.07%
Sleeping reviews              	50.79%
Review Buddies                	0.00%
Ping-pong reviews             	0.00%
At least one of:
- lack_of_review              
- missing_description         
- large_changesets            
- sleeping_reviews            
- review_buddies              
- ping_pong                   	98.65%
All of:
- lack_of_review              
- missing_description         
- large_changesets            
- sleeping_reviews            
- review_buddies              
- ping_pong                   	0.00%


## Wpływ smelli na prawdopodobieństwo wprowadzenia błędu

In [4]:
dbsession = db.getSession()
repo_obj: Repository = dbsession.query(Repository).filter(Repository.full_name == repo).first()

if repo_obj is not None:
    create_functions_in_db(dbsession)
    print()
    print(
        f"Prawdopodobieństwa, że dla przynajmniej jednego z plików zmodyfikowanych "
        f"przez dany PR następna edycja zostanie dokonana przez bug solving PR:")
    print(f"{''.ljust(30)}OK    \t SMELLY\t IMPACT")

    for (name, evaluator) in [
        ("Lack of code review", smells.lack_of_review),
        ("Missing PR description", smells.missing_description),
        ("Large changeset", smells.large_changesets),
        ("Sleeping reviews", smells.sleeping_reviews),
        ("Review Buddies", smells.review_buddies),
        ("Ping-pong reviews", smells.ping_pong)

    ]:
        res = calc_impact(dbsession, repo_obj, evaluator)
        print(f"{name.ljust(30)}{(res[0] * 100):.2f}%\t {(res[1] * 100):.2f}%\t {'+' if res[1]>res[0] else ''}{((res[1]-res[0]) * 100):.2f}%")


Prawdopodobieństwa, że dla przynajmniej jednego z plików zmodyfikowanych przez dany PR następna edycja zostanie dokonana przez bug solving PR:
                              OK    	 SMELLY	 IMPACT
Lack of code review           60.71%	 58.99%	 -1.72%
Missing PR description        60.36%	 55.14%	 -5.21%
Large changeset               57.66%	 81.48%	 +23.83%
Sleeping reviews              57.53%	 60.62%	 +3.09%
Review Buddies                59.10%	 nan%	 nan%
Ping-pong reviews             59.10%	 nan%	 nan%
